# Guidance:
#### How to use the tool to generate XP metrics results:


1.   Activate the function - **test_result**
2.   Prepare the SQL query 
3.   Make sure the query returns the correct structure as requested
4.   Run the function on the SQL output 




In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text
# Scipy helper functions
from scipy.stats import percentileofscore
from scipy import stats
import math

from sqlalchemy import create_engine
from sqlalchemy import text

redshift_endpoint1 = "redshift.sayweee.net"
redshift_user1 = "jiafeng"
redshift_pass1 = "WEEEjiafeng2021!"
port1 = 5439 #whaterver your Redshift portnumber is
dbname1 = "dev"

engine_string = "postgresql+psycopg2://%s:%s@%s:%d/%s" \
% (redshift_user1, redshift_pass1, redshift_endpoint1, port1, dbname1)
engine1 = create_engine(engine_string)

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


* '**welschs_ttest_2**' function: ratio-based test 
  * 表结构和字段名称 --> metric, exp, variation, convert, total
* '**ttest**' function: Mean-based T-test 
  * 表结构和字段名称 --> metric, exp, variation, avg, stddev, sample size
* '**test_result**' function: return results 

In [ ]:
# welschs_ttest_2

## Welsch's t-test function
from scipy.stats import ttest_ind
def welschs_ttest(click_a, num_a, click_b, num_b):
    a_dist = np.zeros(num_a)
    a_dist[:click_a] = 1
    b_dist = np.zeros(num_b)
    b_dist[:click_b] = 1
    zscore, prob = ttest_ind(a_dist, b_dist, equal_var=False)
    #r = print(f"Z-score is {zscore:0.2f}, p-value is {prob:0.3f} (two tailed), {prob/2:0.3f} (one tailed)")
    return prob

def welschs_ttest_2(tb):
    
    # construct the table for welschs_ttest
    df_merge = pd.merge(tb[tb["variation"] == '1'],tb[tb["variation"] == '2'],on=['exp', 'metric'], how ='inner') 
    m = df_merge.select_dtypes(np.number)
    df_merge[m.columns]= m.round().astype('Int64')
    
    # find the p-value & delta
    
    df_merge['p_value'] = 0
    df_merge['significance'] = 0
    for exp in range(len(df_merge)):
        click_a = df_merge.loc[exp, "convert_x"]
        num_a = df_merge.loc[exp, "total_x"]
        click_b = df_merge.loc[exp, "convert_y"]
        num_b = df_merge.loc[exp, "total_y"]
        df_merge.loc[exp, "p_value"] = welschs_ttest(click_a, num_a, click_b, num_b)


    sig_conditions = [
        (df_merge['p_value'] < 0.10) & (df_merge['p_value'] > 0.05),
        (df_merge['p_value'] >= 0.05) & (df_merge['p_value'] < 0.01),
        (df_merge['p_value'] < 0.01)
    ]
    sig_categories = ['90%', '95%', '99%']
    df_merge['significance'] = np.select(sig_conditions, sig_categories)
    df_merge['delta'] = df_merge["convert_x"]/df_merge["total_x"] - df_merge["convert_y"]/df_merge["total_y"]
    df_merge['delta'] = pd.Series(["{0:.2f}%".format(val * 100) for val in df_merge['delta']], index = df_merge.index)
    
    return df_merge

# ttest
def ttest(tb):
    
    # construct the table for t-test
    df_merge = pd.merge(tb[tb["variation"] == '1'],tb[tb["variation"] == '2'],on=['exp', 'metric'], how ='inner') 
    
    # calculate the t-statistic
    df_merge['t-stat'] = (df_merge['avg_x'] - df_merge['avg_y'])
    /
    np.sqrt((df_merge['stddev_x'])**2/df_merge['n_x']+
        (df_merge['stddev_y'])**2/df_merge['n_y'])
    
    # find the p-value & delta
    sig_conditions = [
        (df_merge['t-stat'] < -2.57),
        (df_merge['t-stat'] < -1.96) & (df_merge['t-stat'] > -2.57),
        (df_merge['t-stat'] < -1.645) & (df_merge['t-stat'] > -1.96),
        (df_merge['t-stat'] > 2.57),
        (df_merge['t-stat'] > 1.96) & (df_merge['t-stat'] < 2.57),
        (df_merge['t-stat'] > 1.645) & (df_merge['t-stat'] < 1.96)
    ]
    sig_categories = [ '99% - negative', '95% - negative', '90% - negative','99% - positive', '95% - positive', '90% - positive']
    df_merge['significance'] = np.select(sig_conditions, sig_categories)
    df_merge['delta'] = (df_merge['avg_x'] / df_merge['avg_y']) - 1
    df_merge['delta'] = pd.Series(["{0:.2f}%".format(val * 100) for val in df_merge['delta']], index = df_merge.index)
    
    return df_merge

# return the final datatable
def test_result(df): 
  if len(df.columns) == 5:
    return welschs_ttest_2(df)
  if len(df.columns) == 6:
    return ttest(df)
  else:
    return "invalid SQL output format"

## '**welschs_ttest_2**' function: ratio-based test
*   表结构和字段名称 --> **metric, exp, variation, convert, total**
*  用户输入的sql需要确保表结构和字段名称与以下要求一致以正常运行





In [ ]:
# welschs_ttest_2
sql = """
-- atc session %
with
sess as
    (select distinct e.id
                   , e.name
                   , variation
                   , e.initiator_name
                   , e.status
                   , e.start_time::date                                          as start_date
                   , e.end_time::date                                            as end_date
                   , r.session_token                                             as session_id
                   , case
                         when len(r.device_id) < 10 then 'B_' || r.device_id
                         when len(r.device_id) > 10 then 'A_' || r.device_id end as device_id
                   , r.user_id
                   , case
                         when return_visitor_flag = 0 then 'New Visitor(First_Time_Seen)'
                         when return_visitor_flag = 1 and sess_date <= t.first_purchase_time::Date
                             then 'Return Visitor(Not converted before)'
                         when return_visitor_flag = 1 and t.first_purchase_time is null
                             then 'Return Visitor(Not converted before)'
                         when return_visitor_flag = 1 and sess_date > t.first_purchase_time::Date
                             then 'Return Visitor(Converted before)'
                         else 'Return Visitor(Not converted before)'
            end as return_visitor_flag
     from weee_p01.data_experiment e
              left join weee_p01.experiment_result r on e.id = r.experiment_id
              left join metrics.sessions_details d
                        on d.session_id = r.session_token and d.sess_date >= DATE_ADD('month', -2, CURRENT_DATE)
              left outer join metrics.visitor t on d.device_id = t.device_id
      where e.id in (197, 169, 184)
       and start_date >= DATE_ADD('month', -2, CURRENT_DATE)
)

,
     atc as (
         select o.device_id
              , o.session_id
              , count(distinct target) as atc
         from metrics.event_add_cart o
          where visit_time >= DATE_ADD('month', -2, CURRENT_DATE)
         group by 1,2
)


select 'atc session%' as metric
, id|| ' - ' ||name as exp
, variation
     , count(case when atc > 0 then session_id else null end) as convert
     , count(session_id) as total
from (
         select id
              , name
              , variation
              , sess.session_id
              , coalesce(atc, 0)     as atc

         from sess
                  left join atc o on sess.session_id = o.session_id
     )
group by 1,2,3

""" 
df = pd.read_sql_query(text(sql), engine1)
test_result(df)

,metric,exp,variation_x,convert_x,total_x,variation_y,convert_y,total_y,p_value,significance,delta
0,atc session%,169 - [Activity] 1st App Purchase Coupon for W...,1,69697,147667,2,97717,157193,0.000000,99%,-14.96%
1,atc session%,197 - upsell-68-trade-in-update-discount,1,767096,1630362,2,769784,1639088,0.117191,0,0.09%
2,atc session%,184 - timer_baner_on_checkout_page,1,13210,15150,2,13375,15306,0.619966,0,-0.19%


## '**ttest**' function: Mean-based T-test
*   表结构和字段名称 --> **metric, exp, variation, avg, stddev, sample size**
*  用户输入的sql需要确保表结构和字段名称与以下要求一致以正常运行

In [ ]:
# example - ttest
sql2 = """
with
sess as
    (select distinct e.id
          , e.name
          , variation
          , e.initiator_name
          , e.status
          , e.start_time::date as start_date
          , e.end_time::date as end_date
          , r.session_token as session_id
          , case when len(r.device_id) < 10 then 'B_'|| r.device_id when len(r.device_id) > 10 then 'A_'|| r.device_id end as device_id
          , user_id
     from weee_p01.data_experiment e
              left join weee_p01.experiment_result r on e.id = r.experiment_id
--               join metrics.sessions_details d
--                    on d.session_id = r.session_token and d.sess_date >= DATE_ADD('month', -2, CURRENT_DATE)
     where experiment_id in (197)
        and start_date  >= DATE_ADD('month', -2, CURRENT_DATE)
)

,
     orders as (
         select o.buyer_id
              , o.session_token
              , sum(total)                                             as total
              , sum(gm_margin)                                            gm_margin
              , sum(gm_base)                                              gm_base
              , count(distinct o.order_id)                             as order_cnt
         from metrics.order o
                  left join (select order_id,
                                    1.0 * sum(gm_margin) / sum(gm_base) as gross_margin,
                                    sum(gm_margin)                      as gm_margin,
                                    sum(gm_base)                        as gm_base
                             from metrics.order_product
                             where payment_mode = 'F'
                             group by 1
         ) p on o.order_id = p.order_id

         where payment_mode = 'F'
           and order_day >= DATE_ADD('month', -2, CURRENT_DATE)
         group by 1,2
)


select 'gross margin' as metric, id|| ' - ' ||name as exp, variation
     , avg(gm_margin) as avg, stddev(gm_margin) as stddev, count(user_id) as n
from (
         select id
              , name
              , variation
              , sess.user_id
              , coalesce(sum(total), 0)     as total_booking
              , coalesce(sum(gm_margin), 0)    gm_margin
              , coalesce(sum(gm_base), 0)      gm_base
              , coalesce(sum(order_cnt), 0) as order_cnt

         from sess
                  left join orders o on sess.session_id = o.session_token
         group by 1, 2, 3, 4
         order by 6 desc
     )
group by 1,2,3

""" 
df_mean_based = pd.read_sql_query(text(sql2), engine1)
test_result(df_mean_based)

,metric,exp,variation_x,avg_x,stddev_x,n_x,variation_y,avg_y,stddev_y,n_y,t-stat,significance,delta
0,gross margin,197 - upsell-68-trade-in-update-discount,1,18.59,1865.433212,281518,2,18.56,1900.415358,281661,0.03,0,0.16%


## Quick ratio-based p-value calculation

In [ ]:
welschs_ttest(1956,18196,1891,17948)

0.5102406445323776